# Volvo Trucks Analytics

### Imports and global variables go here

In [3]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import datetime

#The two truck data set paths are set here.
truck1csv = "../data/trucks/truck1.csv"
truck2csv = "../data/trucks/truck2.csv"

#The two column header dictonary paths are set here.
truck1dictcsv = "../data/dictionary/truck1dict.csv"
truck2dictcsv = "../data/dictionary/truck2dict.csv"

#Number of records desired from the data set.
numberOfRecords = 100

#If the number of values that are NOT Na type pass this percentage, the row will be deleted
rowNaNThresholdPercent = 75

#Create column conversion dictionaries from CSV files.
truck1dict={}
truck2dict={}
with open(truck1dictcsv, 'r') as f:
    for row in csv.reader(f):
        truck1dict.update({row[0]:row[1]})
        
with open(truck2dictcsv, 'r') as f:
    for row in csv.reader(f):
        truck2dict.update({row[0]:row[1]})


### Define all cleaning functions here

In [4]:
def readCsv(truck):
    return pd.read_csv(truck, header = [0])
    
def cullUtcCols(truckData):
    noUTC = truckData.drop(["UTC hour", "UTC minute", "UTC second", "UTC month", "UTC day", "UTC year"], axis = 1)
    return noUTC

"""Takes a DataFrame and a conversion dictionary as parameters;
    uses the dictionary to rename all matching columns then returns the changed DataFrame."""
def renameColumns(dataFrame, dictionary):
    return dataFrame.rename(columns = dictionary)

"""Finds threshold of Non-NA type using percentage and deletes rows."""
def removeUnnecessaryRows(dataFrame):
    length = len(dataFrame.columns)
    TValue = (rowNaNThresholdPercent / 100) * length
    print("Threshold value: " + str(int(TValue)))
    return dataFrame.dropna(thresh = int(TValue))

"""Finds columns with all Na types and deletes them."""
def removeUnnecessaryColumns(dataFrame):
    return dataFrame.dropna(axis=1, how='all')

### Cleaning of Truck 1 Data starts here

In [5]:
truck1data = readCsv(truck1csv)
truck1data = cullUtcCols(truck1data)
truck1data = removeUnnecessaryRows(truck1data)
truck1data = renameColumns(truck1data, truck1dict)
truck1data = removeUnnecessaryColumns(truck1data)

# After cleaning, check the shape of the dataframe
truck1data.shape

Threshold value: 33


(1216197, 41)

In [6]:
truck1data.head()

,Time (DateTime),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Steering Wheel Angle (radian),Total Distance (m),Transmission Lube Temperature (C),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),Vehicle Speed (Wheel-Based,CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,-5.660133,246045850.0,27.84375,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,-5.964820,246045850.0,27.84375,NaN,8.167969,18000.0,8.167969,40.0,0.0,6.1116
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,-6.244117,246045850.0,27.84375,NaN,8.117188,18000.0,8.167969,40.0,0.0,6.1116
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,-6.509742,246045850.0,27.84375,NaN,8.117188,18000.0,8.117188,88.0,0.0,6.6672
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,-6.781227,246045850.0,27.84375,NaN,7.820312,18000.0,7.820312,88.0,0.0,6.6672


In [7]:
truck1data

,Time (DateTime),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Steering Wheel Angle (radian),Total Distance (m),Transmission Lube Temperature (C),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),Vehicle Speed (Wheel-Based,CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,-5.660133,246045850.0,27.84375,NaN,8.167969,18000.0,8.167969,40.0,0.000000,6.111600
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,-5.964820,246045850.0,27.84375,NaN,8.167969,18000.0,8.167969,40.0,0.000000,6.111600
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,-6.244117,246045850.0,27.84375,NaN,8.117188,18000.0,8.167969,40.0,0.000000,6.111600
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,-6.509742,246045850.0,27.84375,NaN,8.117188,18000.0,8.117188,88.0,0.000000,6.667200
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,-6.781227,246045850.0,27.84375,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.667200
21,08/07/2019 07:54:05.059,0.005951,13.984894,104.071298,18.373770,0.277523,7.558938,0.126841,-0.023842,0.759136,...,-7.060523,246045850.0,27.84375,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.667200
22,08/07/2019 07:54:05.159,0.007324,13.970703,96.727893,22.276265,0.269894,4.747463,0.132563,0.004768,0.881209,...,-7.322242,246045850.0,27.84375,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.667200
23,08/07/2019 07:54:05.259,NaN,13.959258,90.080682,17.767224,0.271801,5.678264,0.130655,-0.004768,0.774395,...,-7.582008,246045850.0,27.84375,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.667200
24,08/07/2019 07:54:05.359,NaN,13.956054,98.835546,25.068666,0.315671,5.674449,0.140192,0.004768,0.835431,...,-7.855445,246045850.0,27.84375,NaN,7.718750,18000.0,7.718750,88.0,0.000000,6.667200
25,08/07/2019 07:54:05.459,NaN,13.961547,99.321927,22.695888,0.287060,5.663005,0.115396,-0.023842,0.751507,...,-8.101539,246045850.0,27.84375,NaN,7.718750,18000.0,7.718750,88.0,0.000000,6.667200


In [8]:
print(truck1data.describe())

       1730 Automatic Start & Stop (V)  1730 Batteries (V)  \
count                    892537.000000        1.216197e+06   
mean                          0.008484        1.402766e+01   
std                           0.002597        5.379075e-02   
min                           0.000458        1.248569e+01   
25%                           0.007782        1.400092e+01   
50%                           0.009155        1.404395e+01   
75%                           0.010071        1.406134e+01   
max                           0.255894        1.415427e+01   

       4649 Alternator (A)  4649 Battery Out (A)  4649 Trailer (A)  \
count         1.216197e+06          1.216197e+06      1.216197e+06   
mean          4.143630e+01          2.106691e+01      4.329782e-01   
std           3.111164e+01          4.769233e+00      4.261612e-01   
min           2.274548e+00          6.994354e+00     -7.343404e-02   
25%           2.269303e+01          1.806859e+01      1.630808e-01   
50%           2.71562

### Cleaning of Truck 2 Data starts here

In [9]:
truck2data = readCsv(truck2csv)
truck2data = cullUtcCols(truck2data)
truck2data = removeUnnecessaryRows(truck2data)
truck2data = renameColumns(truck2data, truck2dict)
truck2data = removeUnnecessaryColumns(truck2data)

# After cleaning, check the shape of the dataframe
truck2data.shape

Threshold value: 30


(1686292, 37)

In [10]:
truck2data.head()

,Time (DateTime),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),Vehicle Speed (Wheel-Based,Urea Tank Temperature (C),...,Road Inclination (%),Vehicle Weight (kg),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,03/11/2019 14:49:09.038,-35.452125,4.949645,0.555047,-0.430114,-2.283131,NaN,1096.375,106.886719,NaN,...,2.4,18000.0,16.9375,NaN,106.886719,196.0,NaN,75.0,337.5,29.323334
1,03/11/2019 14:49:09.138,-35.480735,5.270085,0.626764,-0.420577,-2.294575,NaN,1098.875,107.101562,NaN,...,2.4,18000.0,16.9375,NaN,107.101562,198.0,NaN,75.0,337.5,29.323334
2,03/11/2019 14:49:09.238,-35.480735,5.197604,0.568780,-62.500000,-2.271687,NaN,1096.750,107.089844,NaN,...,2.4,18000.0,16.9375,NaN,107.089844,198.0,NaN,75.0,337.5,29.323334
3,03/11/2019 14:49:09.338,-35.452125,5.418860,0.638209,-62.500000,-2.290761,411617045.0,1098.375,106.921875,NaN,...,2.4,18000.0,16.9375,NaN,106.921875,200.0,NaN,75.0,337.5,29.323334
4,03/11/2019 14:49:09.438,-35.471199,5.228122,0.590143,-62.500000,-2.283131,411617045.0,1101.750,107.109375,NaN,...,2.4,18000.0,16.9375,NaN,107.109375,200.0,NaN,75.0,337.5,29.323334


In [11]:
truck2data

,Time (DateTime),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),Vehicle Speed (Wheel-Based,Urea Tank Temperature (C),...,Road Inclination (%),Vehicle Weight (kg),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,03/11/2019 14:49:09.038,-35.452125,4.949645,0.555047,-0.430114,-2.283131,NaN,1096.375,106.886719,NaN,...,2.4,18000.0,16.93750,NaN,106.886719,196.0,NaN,75.0,337.500000,29.323334
1,03/11/2019 14:49:09.138,-35.480735,5.270085,0.626764,-0.420577,-2.294575,NaN,1098.875,107.101562,NaN,...,2.4,18000.0,16.93750,NaN,107.101562,198.0,NaN,75.0,337.500000,29.323334
2,03/11/2019 14:49:09.238,-35.480735,5.197604,0.568780,-62.500000,-2.271687,NaN,1096.750,107.089844,NaN,...,2.4,18000.0,16.93750,NaN,107.089844,198.0,NaN,75.0,337.500000,29.323334
3,03/11/2019 14:49:09.338,-35.452125,5.418860,0.638209,-62.500000,-2.290761,411617045.0,1098.375,106.921875,NaN,...,2.4,18000.0,16.93750,NaN,106.921875,200.0,NaN,75.0,337.500000,29.323334
4,03/11/2019 14:49:09.438,-35.471199,5.228122,0.590143,-62.500000,-2.283131,411617045.0,1101.750,107.109375,NaN,...,2.4,18000.0,16.93750,NaN,107.109375,200.0,NaN,75.0,337.500000,29.323334
5,03/11/2019 14:49:09.538,-35.452125,5.544747,0.529870,-62.500000,-2.283131,411617045.0,1096.875,107.238281,NaN,...,2.4,18000.0,16.93750,NaN,107.238281,200.0,NaN,75.0,337.500000,29.323334
6,03/11/2019 14:49:09.638,-35.442588,5.285344,0.580987,-62.500000,-2.256428,411617045.0,1102.750,107.300781,NaN,...,2.4,18000.0,16.93750,NaN,107.300781,202.0,NaN,75.0,337.500000,29.323334
7,03/11/2019 14:49:09.738,-35.461662,5.151827,0.558862,-62.500000,-2.264057,411617045.0,1103.000,107.171875,NaN,...,2.4,18000.0,16.93750,NaN,107.171875,202.0,NaN,75.0,337.500000,29.323334
8,03/11/2019 14:49:09.838,-35.471199,5.010681,0.548180,-62.500000,-2.241169,411617045.0,1102.125,107.316406,NaN,...,2.4,18000.0,16.93750,NaN,107.316406,202.0,NaN,75.0,337.500000,29.323334
9,03/11/2019 14:49:09.938,-35.461662,5.350195,0.574121,-62.500000,-2.248798,411617045.0,1105.125,107.316406,NaN,...,2.4,18000.0,16.93750,NaN,107.316406,202.0,NaN,75.0,337.500000,29.323334


In [12]:
print(truck2data.describe())

       1115 Alternator (A)  1115 Battery Out (A)  1115 Fridge (A)  \
count         1.686292e+06          1.686292e+06     1.686292e+06   
mean         -3.701792e+01          5.442930e+00     2.732780e-01   
std           1.265965e+00          1.263030e+00     3.271822e-01   
min          -4.000122e+01          1.699474e+00    -1.194018e-01   
25%          -3.794125e+01          4.507134e+00    -5.302510e-02   
50%          -3.706386e+01          5.205234e+00     3.994049e-01   
75%          -3.622461e+01          6.273365e+00     5.855650e-01   
max          -3.255291e+01          1.323148e+01     1.666667e+00   

       1115 Trailer (A)  1115 Inverter (A)  Total Distance (m)  \
count      1.686292e+06       1.686292e+06        1.686284e+06   
mean      -2.919934e+01      -2.279224e+00        4.125769e+08   
std        3.091849e+01       2.589300e-01        1.100496e+06   
min       -6.250000e+01      -9.905013e+00        4.105175e+08   
25%       -6.250000e+01      -2.313649e+00      

### Concatenation of Truck1 and Truck2 Data

In [13]:
df = pd.concat([truck1data, truck2data], sort=False)

In [14]:
df.head()

,Time (DateTime),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,1115 Trailer (A),1115 Inverter (A),Steering Wheel Angle,Engine Output Torque HiRes (%),Coolant Temperature (C),Exhaust Back Pressure (kPa),Pressure of Intake Manifold (kPa),Temperature of Pre-Combustion Air (C),Road Inclination (%),Altitude (m)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.describe()

,1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),4649 Battery Voltage (V),...,1115 Trailer (A),1115 Inverter (A),Steering Wheel Angle,Engine Output Torque HiRes (%),Coolant Temperature (C),Exhaust Back Pressure (kPa),Pressure of Intake Manifold (kPa),Temperature of Pre-Combustion Air (C),Road Inclination (%),Altitude (m)
count,892537.000000,1.216197e+06,1.216197e+06,1.216197e+06,1.216197e+06,1.216197e+06,1.216197e+06,1.216197e+06,1.216197e+06,1.216197e+06,...,1.686292e+06,1.686292e+06,1.686275e+06,1686292.000,1.686292e+06,905467.000000,1.686292e+06,1.686292e+06,1.686292e+06,1.686229e+06
mean,0.008484,1.402766e+01,4.143630e+01,2.106691e+01,4.329782e-01,1.336951e+01,-2.104773e-02,-7.023623e-02,9.704267e-02,1.386995e+01,...,-2.919934e+01,-2.279224e+00,4.382393e-02,1.875,7.922576e+01,49.450935,6.370837e+01,3.041599e+01,1.315044e+00,2.204394e+02
std,0.002597,5.379075e-02,3.111164e+01,4.769233e+00,4.261612e-01,1.643317e+01,5.838831e-02,4.943761e-02,5.352165e+00,2.840073e-01,...,3.091849e+01,2.589300e-01,1.094988e+00,0.000,2.093961e+00,38.689144,5.674264e+01,8.556959e+00,1.720988e+00,1.815942e+02
min,0.000458,1.248569e+01,2.274548e+00,6.994354e+00,-7.343404e-02,-1.040265e+02,-4.396506e-01,-2.431907e-01,-3.952468e+01,1.214054e+01,...,-6.250000e+01,-9.905013e+00,-1.444920e+01,1.875,6.300000e+01,0.000000,0.000000e+00,1.600000e+01,-6.400000e+00,-4.223000e+02
25%,0.007782,1.400092e+01,2.269303e+01,1.806859e+01,1.630808e-01,1.276608e+01,-7.152667e-02,-1.096742e-01,-4.234379e-01,1.380865e+01,...,-6.250000e+01,-2.313649e+00,-2.031016e-01,1.875,7.800000e+01,11.500000,4.000000e+00,2.500000e+01,0.000000e+00,9.640000e+01
50%,0.009155,1.404395e+01,2.715629e+01,2.039178e+01,1.840620e-01,1.591325e+01,-1.812009e-02,-8.106355e-02,-4.196231e-02,1.401144e+01,...,-5.712596e-01,-2.267872e+00,-1.073984e-01,1.875,7.900000e+01,44.000000,5.200000e+01,2.900000e+01,1.200000e+00,1.795000e+02
75%,0.010071,1.406134e+01,4.763199e+01,2.308881e+01,6.876097e-01,1.875525e+01,2.765698e-02,-4.291600e-02,1.792935e-01,1.404532e+01,...,-4.510948e-01,-2.222095e+00,-2.927344e-02,1.875,8.100000e+01,83.500000,1.100000e+02,3.400000e+01,2.400000e+00,3.093000e+02
max,0.255894,1.415427e+01,2.336585e+02,7.557603e+01,3.821431e+00,1.250000e+02,4.911498e-01,2.050431e-01,2.500000e+02,1.413001e+01,...,6.911383e+00,-1.935988e+00,1.310159e+01,1.875,1.000000e+02,124.500000,1.980000e+02,7.100000e+01,7.800000e+00,7.996000e+02


In [16]:
print(df.describe())

       1730 Automatic Start & Stop (V)  1730 Batteries (V)  \
count                    892537.000000        1.216197e+06   
mean                          0.008484        1.402766e+01   
std                           0.002597        5.379075e-02   
min                           0.000458        1.248569e+01   
25%                           0.007782        1.400092e+01   
50%                           0.009155        1.404395e+01   
75%                           0.010071        1.406134e+01   
max                           0.255894        1.415427e+01   

       4649 Alternator (A)  4649 Battery Out (A)  4649 Trailer (A)  \
count         1.216197e+06          1.216197e+06      1.216197e+06   
mean          4.143630e+01          2.106691e+01      4.329782e-01   
std           3.111164e+01          4.769233e+00      4.261612e-01   
min           2.274548e+00          6.994354e+00     -7.343404e-02   
25%           2.269303e+01          1.806859e+01      1.630808e-01   
50%           2.71562

### Group Truck1 by Weight

In [17]:
byWeight = truck1data.groupby('Vehicle Weight (kg)')

In [18]:
byWeight.head()

,Time (DateTime),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,Steering Wheel Angle (radian),Total Distance (m),Transmission Lube Temperature (C),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Weight (kg),Vehicle Speed (Wheel-Based,CPU Load (%),Altitude(m),Speed (km/hr)
16,08/07/2019 07:54:04.559,0.006409,13.946441,97.500381,27.971695,0.277523,1.337072,0.145914,-0.033379,0.843061,...,-5.660133,246045850.0,27.84375,NaN,8.167969,18000.0,8.167969,40.0,0.000000,6.111600
17,08/07/2019 07:54:04.659,0.007782,13.953765,96.985389,22.375448,0.275616,5.640116,0.134470,-0.004768,0.774395,...,-5.964820,246045850.0,27.84375,NaN,8.167969,18000.0,8.167969,40.0,0.000000,6.111600
18,08/07/2019 07:54:04.759,0.007782,13.956970,98.330091,18.987945,0.273709,1.287480,0.126841,-0.014305,0.736248,...,-6.244117,246045850.0,27.84375,NaN,8.117188,18000.0,8.167969,40.0,0.000000,6.111600
19,08/07/2019 07:54:04.859,0.007324,13.953307,94.076638,19.071870,0.277523,6.105516,0.136378,-0.071527,0.774395,...,-6.509742,246045850.0,27.84375,NaN,8.117188,18000.0,8.117188,88.0,0.000000,6.667200
20,08/07/2019 07:54:04.959,0.007324,13.957885,96.365492,21.677348,0.264172,3.751812,0.128748,-0.071527,0.759136,...,-6.781227,246045850.0,27.84375,NaN,7.820312,18000.0,7.820312,88.0,0.000000,6.667200
206,08/07/2019 07:54:23.559,NaN,13.886931,169.141489,21.204318,0.315671,2.527276,0.021935,0.090600,-2.162966,...,0.894555,246045905.0,28.12500,47347.355469,12.039062,16000.0,12.039062,52.0,146.800003,12.038000
207,08/07/2019 07:54:23.659,NaN,13.880980,161.960212,16.630426,0.311856,4.720760,0.012398,0.119211,-1.903563,...,0.894555,246045905.0,28.12500,47313.339844,12.039062,16000.0,12.039062,52.0,146.800003,12.038000
208,08/07/2019 07:54:23.759,NaN,13.886473,163.705463,16.000992,0.302319,3.732738,0.025750,0.090600,-2.132448,...,0.894555,246045905.0,28.12500,47269.273438,11.921875,16000.0,12.039062,52.0,146.800003,12.038000
209,08/07/2019 07:54:23.859,NaN,13.889220,168.636034,15.299077,0.317578,4.221027,0.023842,0.100137,-2.567330,...,0.894555,246045905.0,28.12500,47260.621094,11.921875,16000.0,11.921875,51.0,146.600006,11.852800
210,08/07/2019 07:54:23.959,NaN,13.886015,165.050164,18.968872,0.306134,4.957275,0.031472,0.119211,-2.162966,...,0.894555,246045905.0,28.15625,47291.757812,11.937500,16000.0,11.921875,51.0,146.600006,11.852800


In [19]:
print(byWeight.describe())

                    1730 Automatic Start & Stop (V)                      \
                                              count      mean       std   
Vehicle Weight (kg)                                                       
8000.0                                      29514.0  0.009671  0.002733   
16000.0                                    279997.0  0.007672  0.002783   
17000.0                                    138055.0  0.009924  0.001608   
18000.0                                       584.0  0.009661  0.001809   
19000.0                                     16879.0  0.010159  0.000960   
20000.0                                      1179.0  0.009690  0.001031   
21000.0                                       335.0  0.009598  0.002050   
22000.0                                     49465.0  0.007808  0.001745   
23000.0                                     49350.0  0.005119  0.003097   
24000.0                                    325181.0  0.008985  0.002071   
25000.0                  

### Group Truck2 by Weight

In [20]:
byWeight2 = truck2data.groupby('Vehicle Weight (kg)', axis=0)

In [21]:
byWeight2.head()

,Time (DateTime),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),Vehicle Speed (Wheel-Based,Urea Tank Temperature (C),...,Road Inclination (%),Vehicle Weight (kg),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,03/11/2019 14:49:09.038,-35.452125,4.949645,0.555047,-0.430114,-2.283131,NaN,1096.375,106.886719,NaN,...,2.4,18000.0,16.93750,NaN,106.886719,196.0,NaN,75.0,337.500000,2.932333e+01
1,03/11/2019 14:49:09.138,-35.480735,5.270085,0.626764,-0.420577,-2.294575,NaN,1098.875,107.101562,NaN,...,2.4,18000.0,16.93750,NaN,107.101562,198.0,NaN,75.0,337.500000,2.932333e+01
2,03/11/2019 14:49:09.238,-35.480735,5.197604,0.568780,-62.500000,-2.271687,NaN,1096.750,107.089844,NaN,...,2.4,18000.0,16.93750,NaN,107.089844,198.0,NaN,75.0,337.500000,2.932333e+01
3,03/11/2019 14:49:09.338,-35.452125,5.418860,0.638209,-62.500000,-2.290761,411617045.0,1098.375,106.921875,NaN,...,2.4,18000.0,16.93750,NaN,106.921875,200.0,NaN,75.0,337.500000,2.932333e+01
4,03/11/2019 14:49:09.438,-35.471199,5.228122,0.590143,-62.500000,-2.283131,411617045.0,1101.750,107.109375,NaN,...,2.4,18000.0,16.93750,NaN,107.109375,200.0,NaN,75.0,337.500000,2.932333e+01
13960,03/11/2019 03:25:17.389,-37.931716,8.123522,0.758755,-0.422484,-2.244984,410517455.0,1150.500,112.195312,NaN,...,2.2,19000.0,12.21875,112.0,111.894531,220.0,NaN,85.0,27.000000,3.102100e+01
13961,03/11/2019 03:25:17.489,-37.941253,7.448310,0.740444,-0.435836,-2.260243,410517455.0,1150.500,112.195312,NaN,...,2.4,19000.0,12.21875,112.0,111.984375,220.0,NaN,85.0,27.000000,3.102100e+01
13962,03/11/2019 03:25:17.589,-37.931716,7.085908,0.800717,-0.449187,-2.267872,410517455.0,1148.875,111.984375,NaN,...,2.2,19000.0,12.21875,112.0,112.121094,220.0,NaN,85.0,27.000000,3.102100e+01
13963,03/11/2019 03:25:17.689,-37.931716,7.440681,0.748836,-0.445373,-2.271687,410517455.0,1149.000,111.984375,NaN,...,2.2,19000.0,12.21875,112.0,112.093750,222.0,NaN,85.0,27.000000,3.102100e+01
13964,03/11/2019 03:25:17.789,-37.941253,8.508812,0.790036,-0.420577,-2.203021,410517455.0,1149.625,111.917969,NaN,...,2.2,19000.0,12.21875,112.0,111.917969,220.0,NaN,85.0,27.000000,3.102100e+01


In [22]:
print(byWeight2.describe())

                    1115 Alternator (A)                                  \
                                  count       mean       std        min   
Vehicle Weight (kg)                                                       
8000.0                          86339.0 -35.692719  1.710949 -38.894942   
16000.0                         19356.0 -38.273179  0.340796 -38.723278   
17000.0                         96085.0 -36.767462  0.834981 -38.008011   
18000.0                        358430.0 -36.875129  0.767383 -38.360876   
19000.0                        167268.0 -37.882754  0.870605 -39.200122   
20000.0                         69905.0 -37.552775  0.725278 -38.847257   
21000.0                          3230.0 -37.061284  0.122712 -37.283207   
22000.0                            82.0 -36.973607  0.011032 -36.997101   
23000.0                            10.0 -36.858816  0.010301 -36.882658   
24000.0                            11.0 -36.860983  0.007498 -36.873121   
25000.0                  

In [23]:
byWeight2 = truck2data.groupby('Vehicle Weight (kg)')

for key, item in byWeight2:
    print(byWeight2.get_group(key), "\n\n")

                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
142497   03/12/2019 19:22:06.926           -38.103380              3.435187   
142498   03/12/2019 19:22:07.026           -38.084306              3.454261   
142499   03/12/2019 19:22:07.126           -38.103380              3.282597   
142500   03/12/2019 19:22:07.226           -38.093843              3.534371   
142501   03/12/2019 19:22:07.326           -38.122454              3.454261   
142502   03/12/2019 19:22:07.426           -38.084306              3.919661   
142503   03/12/2019 19:22:07.526           -38.112917              3.824292   
142504   03/12/2019 19:22:07.626           -38.093843              3.809033   
142505   03/12/2019 19:22:07.726           -38.093843              3.885328   
142506   03/12/2019 19:22:07.826           -38.112917              4.121843   
142507   03/12/2019 19:22:07.926           -38.103380              3.614481   
142508   03/12/2019 19:22:08.026           -38.10338

                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
0        03/11/2019 14:49:09.038           -35.452125              4.949645   
1        03/11/2019 14:49:09.138           -35.480735              5.270085   
2        03/11/2019 14:49:09.238           -35.480735              5.197604   
3        03/11/2019 14:49:09.338           -35.452125              5.418860   
4        03/11/2019 14:49:09.438           -35.471199              5.228122   
5        03/11/2019 14:49:09.538           -35.452125              5.544747   
6        03/11/2019 14:49:09.638           -35.442588              5.285344   
7        03/11/2019 14:49:09.738           -35.461662              5.151827   
8        03/11/2019 14:49:09.838           -35.471199              5.010681   
9        03/11/2019 14:49:09.938           -35.461662              5.350195   
10       03/11/2019 14:49:10.038           -35.442588              4.873350   
11       03/11/2019 14:49:10.138           -35.48073

                Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
160785  03/11/2019 06:53:40.080           -36.958953              4.789425   
160786  03/11/2019 06:53:40.180           -36.958953              4.610132   
160787  03/11/2019 06:53:40.280           -36.968490              5.430304   
160788  03/11/2019 06:53:40.380           -36.968490              4.724575   
160789  03/11/2019 06:53:40.480           -36.987564              5.521859   
160790  03/11/2019 06:53:40.580           -36.968490              5.254826   
160791  03/11/2019 06:53:40.680           -36.987564              4.747463   
160792  03/11/2019 06:53:40.780           -36.958953              4.747463   
160793  03/11/2019 06:53:40.880           -36.958953              5.281529   
160794  03/11/2019 06:53:40.980           -36.958953              6.124590   
160795  03/11/2019 06:53:41.080           -36.978027              5.075532   
160796  03/11/2019 06:53:41.180           -36.978027            

[56455 rows x 37 columns] 


                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
324972   03/12/2019 15:27:31.262           -34.097887              6.029221   
324973   03/12/2019 15:27:31.362           -34.097887              5.331121   
324974   03/12/2019 15:27:31.462           -34.078813              5.750744   
324975   03/12/2019 15:27:31.562           -34.069276              5.926223   
324976   03/12/2019 15:27:31.662           -34.116960              5.701152   
324977   03/12/2019 15:27:31.762           -34.078813              5.552377   
324978   03/12/2019 15:27:31.862           -34.088350              5.365454   
324979   03/12/2019 15:27:31.962           -34.069276              5.315862   
324980   03/12/2019 15:27:32.062           -34.088350              4.152361   
324981   03/12/2019 15:27:32.162           -34.088350              4.438468   
324982   03/12/2019 15:27:32.262           -34.088350              5.762188   
324983   03/12/2019 15:

                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
325107   03/12/2019 15:27:44.762           -34.097887              4.491875   
325108   03/12/2019 15:27:44.862           -34.097887              4.736019   
325109   03/12/2019 15:27:44.962           -34.088350              4.701686   
325110   03/12/2019 15:27:45.062           -34.088350              5.495155   
325111   03/12/2019 15:27:45.162           -34.069276              5.121309   
325112   03/12/2019 15:27:45.262           -34.078813              4.892424   
325113   03/12/2019 15:27:45.362           -34.088350              5.006867   
325114   03/12/2019 15:27:45.462           -34.078813              4.907683   
325115   03/12/2019 15:27:45.562           -34.078813              4.907683   
325116   03/12/2019 15:27:45.662           -34.050202              6.277180   
325117   03/12/2019 15:27:45.762           -34.078813              4.812314   
325118   03/12/2019 15:27:45.862           -34.05973

                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
134224   03/12/2019 10:46:54.608           -36.653773              5.704967   
134225   03/12/2019 10:46:54.708           -36.644236              4.991608   
134226   03/12/2019 10:46:54.808           -36.672847              5.106050   
134227   03/12/2019 10:46:54.908           -36.653773              5.025940   
134228   03/12/2019 10:46:55.008           -36.663310              6.128405   
134229   03/12/2019 10:46:55.108           -36.663310              6.040665   
134230   03/12/2019 10:46:55.208           -36.663310              5.128939   
134231   03/12/2019 10:46:55.308           -36.653773              5.022126   
134232   03/12/2019 10:46:55.408           -36.663310              5.262455   
134233   03/12/2019 10:46:55.508           -36.634699              5.251011   
134234   03/12/2019 10:46:55.608           -36.663310              5.354009   
134235   03/12/2019 10:46:55.708           -36.64423

                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
544478   03/12/2019 05:07:39.060           -39.915389              4.255360   
544479   03/12/2019 05:07:39.160           -39.896315              4.911498   
544480   03/12/2019 05:07:39.260           -39.915389              3.740368   
544481   03/12/2019 05:07:39.360           -39.905852              3.568704   
544482   03/12/2019 05:07:39.460           -39.915389              3.839551   
544483   03/12/2019 05:07:39.560           -39.905852              3.751812   
544484   03/12/2019 05:07:39.660           -39.896315              3.961624   
544485   03/12/2019 05:07:39.760           -39.924926              4.015030   
544486   03/12/2019 05:07:39.860           -39.905852              3.892958   
544487   03/12/2019 05:07:39.960           -39.915389              3.686961   
544488   03/12/2019 05:07:40.060           -39.924926              4.068437   
544489   03/12/2019 05:07:40.160           -39.90585

                 Time (DateTime)  1115 Alternator (A)  1115 Battery Out (A)  \
544546   03/12/2019 05:07:45.860           -39.924926              4.049363   
544547   03/12/2019 05:07:45.960           -39.934463              4.873350   
544548   03/12/2019 05:07:46.060           -39.924926              3.931106   
544549   03/12/2019 05:07:46.160           -39.905852              4.064622   
544550   03/12/2019 05:07:46.260           -39.905852              3.919661   
544551   03/12/2019 05:07:46.360           -39.924926              4.415579   
544552   03/12/2019 05:07:46.460           -39.924926              5.258640   
544553   03/12/2019 05:07:46.560           -39.924926              3.889143   
544554   03/12/2019 05:07:46.660           -39.924926              3.782330   
544555   03/12/2019 05:07:46.760           -39.934463              3.862440   
544556   03/12/2019 05:07:46.860           -39.943999              3.950179   
544557   03/12/2019 05:07:46.960           -39.92492

In [24]:
df = truck1data

This sums all the data around the grouping by 'Vehicle Weight'

In [25]:
df.groupby(['Vehicle Weight (kg)'], as_index=False).sum()

,Vehicle Weight (kg),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,RoadInclinationVP15,Steering Wheel Angle (radian),Total Distance (m),Transmission Lube Temperature (C),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Speed (Wheel-Based,CPU Load (%),Altitude(m),Speed (km/hr)
0,8000.0,285.431898,5.619897e+05,1.861829e+06,7.518836e+05,3687.855726,2.928542e+05,-1168.957427,-2445.277333,45784.153505,...,72276.4,-1856.866766,9.882293e+12,2.535574e+06,1.576351e+09,1.815810e+06,1.815836e+06,1943254.0,9.586676e+06,1.805107e+06
1,16000.0,2148.139536,6.059402e+06,1.906044e+07,9.056271e+06,125306.127452,5.429798e+06,-15574.522202,-28803.916800,243381.746394,...,587160.6,7011.612352,1.063787e+14,3.209450e+07,2.067661e+10,3.261190e+07,3.261176e+07,20528263.0,9.953091e+07,3.252260e+07
2,17000.0,1369.992177,1.987305e+06,3.645040e+06,2.896471e+06,30295.653086,1.833518e+06,-9050.455863,-12415.827420,2474.319066,...,214112.6,6312.996141,3.489259e+13,1.086160e+07,7.512862e+09,1.165080e+07,1.165084e+07,6766582.0,3.372880e+07,1.162552e+07
3,18000.0,5.642023,7.125106e+05,4.847343e+06,9.707795e+05,13459.036202,8.666253e+05,1518.182078,814.884222,-77446.314946,...,87562.0,552.654977,1.255929e+13,3.478031e+06,2.342844e+09,4.325510e+06,4.325511e+06,2320604.0,1.069256e+07,4.309658e+06
4,19000.0,171.478136,2.381788e+05,3.875295e+05,3.616468e+05,17481.656748,2.279233e+05,-757.330053,-1203.507668,694.956893,...,13385.0,-1916.459234,4.181082e+12,1.284760e+06,8.750001e+08,1.070087e+06,1.070045e+06,807638.0,3.810484e+06,1.064600e+06
5,20000.0,11.424124,1.663562e+04,2.245239e+04,1.984827e+04,590.450713,1.597598e+04,52.894446,-89.641985,93.801022,...,-537.8,-1317.542680,2.911985e+11,8.906372e+04,1.934886e+07,8.750000e+01,8.726172e+01,56663.0,3.227927e+05,3.704000e+01
6,21000.0,3.215381,5.361445e+03,9.116808e+03,6.430068e+03,91.891737,4.721874e+03,-44.796674,-41.504540,12.863355,...,874.0,-31.104328,9.414212e+10,3.009266e+04,6.262960e+06,1.892930e+02,1.878633e+02,18992.0,9.765530e+04,1.129720e+02
7,22000.0,386.242146,6.972634e+05,1.622111e+06,1.311262e+06,14115.796903,7.026624e+05,34.191653,-4579.222934,3247.302968,...,53518.8,2999.343539,1.222680e+13,3.730309e+06,2.363720e+09,3.123599e+06,3.123677e+06,2421427.0,1.103873e+07,3.109148e+06
8,23000.0,252.621180,9.359827e+05,3.039674e+06,1.489743e+06,29006.916152,1.030774e+06,1077.702754,-4545.357443,-16734.019989,...,70902.8,2969.661289,1.640565e+13,4.807634e+06,3.200141e+09,3.947247e+06,3.947300e+06,3264685.0,1.542357e+07,3.933325e+06
9,24000.0,2921.829612,5.812903e+06,1.581831e+07,8.714980e+06,291528.354127,5.824739e+06,-1737.849050,-31948.896582,-83459.193560,...,511523.8,11697.092289,1.018310e+14,3.078374e+07,2.374689e+10,3.247290e+07,3.247286e+07,20044261.0,9.898163e+07,3.230005e+07


In [26]:
df2 = truck2data

This groups by the 'Vehicle Weight' then uses the mean as the new values in the rest of the columns

In [27]:
df2.groupby(['Vehicle Weight (kg)'], as_index=False).mean()

,Vehicle Weight (kg),1115 Alternator (A),1115 Battery Out (A),1115 Fridge (A),1115 Trailer (A),1115 Inverter (A),Total Distance (m),Engine Speed (rpm),Vehicle Speed (Wheel-Based,Urea Tank Temperature (C),...,Transmission Lube Temperature (C),Road Inclination (%),Outside Air Temperature (C),Cruise Control Speed (km/hr),Vehicle Speed (km/hr),Boost Pressure (kPa),Turbo Speed (rpm),CPU Load (%),Altitude (m),Speed (km/hr)
0,8000.0,-35.692719,4.615587,0.337674,-53.678082,-2.174727,4.136591e+08,654.934601,3.955776,19.768743,...,77.004530,-0.435544,18.069960,NaN,3.955642,93.043468,19258.520966,59.665169,150.382341,1.090766
1,16000.0,-38.273179,5.465820,0.084065,-0.910419,-2.295367,4.123853e+08,714.711963,14.969260,16.000000,...,78.911013,0.832806,9.765985,103.000000,14.965997,106.161914,24069.025471,59.996435,5.960421,4.169823
2,17000.0,-36.767462,5.240357,0.297139,-51.609074,-2.220430,4.118324e+08,1139.223742,107.521365,17.409207,...,83.864726,1.731735,16.113638,111.577260,107.521124,163.732487,57516.294456,58.824218,259.940457,29.729715
3,18000.0,-36.875129,5.338060,0.238629,-29.245123,-2.268138,4.113586e+08,1063.449810,89.443117,15.128973,...,83.028535,1.790975,12.795059,110.390205,89.442681,161.932132,54176.098946,57.928321,274.631913,24.733662
4,19000.0,-37.882754,5.479267,0.349616,-28.680825,-2.305169,4.115231e+08,1113.321193,99.214402,15.195384,...,81.702201,1.866688,11.596259,105.833240,99.214447,170.525934,57290.826739,58.423351,100.538198,27.424575
5,20000.0,-37.552775,4.826720,0.290199,-18.578878,-2.265395,4.112384e+08,1146.534236,107.506083,15.235824,...,81.943431,1.975965,10.718096,111.656111,107.506580,170.357313,57779.579222,57.468164,154.834523,29.689566
6,21000.0,-37.061284,5.661872,0.573403,-37.335995,-2.230323,4.115410e+08,1143.393034,95.441905,16.642105,...,78.576567,1.627430,12.128270,NaN,95.452359,189.886687,65787.175289,55.886378,117.909690,26.144911
7,22000.0,-36.973607,5.414859,0.629900,-0.458259,-2.259405,4.108127e+08,1196.809451,71.182974,15.000000,...,78.671494,1.626829,10.304116,NaN,71.264910,221.073171,73076.986947,61.878049,119.848781,18.282854
8,23000.0,-36.858816,5.663767,-0.062257,-0.855840,-2.221714,4.123438e+08,1254.337500,58.481641,16.000000,...,79.500000,2.700000,10.562500,NaN,58.481641,226.000000,74694.892969,61.200000,12.870000,14.857156
9,24000.0,-36.860983,5.083162,-0.048794,-0.877827,-2.290761,4.123438e+08,1202.090909,56.048295,16.000000,...,79.488636,2.745455,10.562500,NaN,56.112926,222.727273,73252.679688,55.818182,12.800000,13.992889


In [28]:
df.groupby(['Vehicle Weight (kg)'], as_index=False).mean()

,Vehicle Weight (kg),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,RoadInclinationVP15,Steering Wheel Angle (radian),Total Distance (m),Transmission Lube Temperature (C),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Speed (Wheel-Based,CPU Load (%),Altitude(m),Speed (km/hr)
0,8000.0,0.009671,14.043424,46.524783,18.788636,0.092155,7.318062,-0.029211,-0.061104,1.144089,...,1.806097,-0.046401,2.469462e+08,63.360827,40229.454977,45.374841,45.375491,48.559498,239.559108,46.430032
1,16000.0,0.007672,14.014616,44.084353,20.945990,0.289817,12.558423,-0.036022,-0.066620,0.562911,...,1.358027,0.016226,2.460402e+08,74.230444,53468.556699,75.427127,75.426809,47.479232,230.202190,75.223015
2,17000.0,0.009924,14.044357,25.759633,20.469468,0.214101,12.957542,-0.063960,-0.087743,0.017486,...,1.513142,0.044614,2.465873e+08,76.759321,56002.607032,82.336678,82.336901,47.819692,238.362716,82.157967
3,18000.0,0.009661,13.962856,94.991926,19.024075,0.263753,16.982996,0.029751,0.015969,-1.517692,...,1.715926,0.010830,2.461206e+08,68.157927,60309.524053,84.765731,84.765746,45.476180,209.538841,84.455076
4,19000.0,0.010159,14.088416,22.922604,21.391623,1.034050,13.481798,-0.044797,-0.071188,0.041107,...,0.791731,-0.113360,2.473135e+08,75.994296,51756.779908,63.296274,63.293811,47.772270,225.392393,62.971702
5,20000.0,0.009690,14.109940,19.043585,16.834838,0.500806,13.550450,0.044864,-0.076032,0.079560,...,-0.456149,-1.117509,2.469877e+08,75.541746,16411.244329,0.074215,0.074013,48.060221,273.785158,0.031416
6,21000.0,0.009598,14.035197,23.865989,16.832638,0.240554,12.360926,-0.117269,-0.108651,0.033674,...,2.287958,-0.081425,2.464453e+08,78.776587,16395.182098,0.495531,0.491789,49.717277,255.642147,0.295738
7,22000.0,0.007808,14.029445,32.638046,26.383538,0.284020,14.138076,0.000688,-0.092137,0.065338,...,1.076837,0.060912,2.460121e+08,75.056526,47710.482928,62.849083,62.850641,48.720865,222.107231,62.558301
8,23000.0,0.005119,14.023054,45.540919,22.319582,0.434587,15.443225,0.016146,-0.068099,-0.250712,...,1.062278,0.044513,2.457923e+08,72.028790,48192.713733,59.138337,59.139118,48.912070,231.078640,58.929745
9,24000.0,0.008985,14.039709,38.205425,21.048999,0.704119,14.068296,-0.004197,-0.077165,-0.201576,...,1.235466,0.028252,2.459489e+08,74.350943,59184.630333,78.430697,78.430600,48.412230,239.067010,78.030373


In [29]:
dftest = truck1data

In [30]:
dftest.groupby(['Vehicle Weight (kg)'], as_index=False).mean()

,Vehicle Weight (kg),1730 Automatic Start & Stop (V),1730 Batteries (V),4649 Alternator (A),4649 Battery Out (A),4649 Trailer (A),4649 Inverter (A),4649 Fridge (A),4649 Battery Bank (A),4649 Battery Separator (A),...,RoadInclinationVP15,Steering Wheel Angle (radian),Total Distance (m),Transmission Lube Temperature (C),Turbo Speed (rpm),Vehicle Speed (km/hr),Vehicle Speed (Wheel-Based,CPU Load (%),Altitude(m),Speed (km/hr)
0,8000.0,0.009671,14.043424,46.524783,18.788636,0.092155,7.318062,-0.029211,-0.061104,1.144089,...,1.806097,-0.046401,2.469462e+08,63.360827,40229.454977,45.374841,45.375491,48.559498,239.559108,46.430032
1,16000.0,0.007672,14.014616,44.084353,20.945990,0.289817,12.558423,-0.036022,-0.066620,0.562911,...,1.358027,0.016226,2.460402e+08,74.230444,53468.556699,75.427127,75.426809,47.479232,230.202190,75.223015
2,17000.0,0.009924,14.044357,25.759633,20.469468,0.214101,12.957542,-0.063960,-0.087743,0.017486,...,1.513142,0.044614,2.465873e+08,76.759321,56002.607032,82.336678,82.336901,47.819692,238.362716,82.157967
3,18000.0,0.009661,13.962856,94.991926,19.024075,0.263753,16.982996,0.029751,0.015969,-1.517692,...,1.715926,0.010830,2.461206e+08,68.157927,60309.524053,84.765731,84.765746,45.476180,209.538841,84.455076
4,19000.0,0.010159,14.088416,22.922604,21.391623,1.034050,13.481798,-0.044797,-0.071188,0.041107,...,0.791731,-0.113360,2.473135e+08,75.994296,51756.779908,63.296274,63.293811,47.772270,225.392393,62.971702
5,20000.0,0.009690,14.109940,19.043585,16.834838,0.500806,13.550450,0.044864,-0.076032,0.079560,...,-0.456149,-1.117509,2.469877e+08,75.541746,16411.244329,0.074215,0.074013,48.060221,273.785158,0.031416
6,21000.0,0.009598,14.035197,23.865989,16.832638,0.240554,12.360926,-0.117269,-0.108651,0.033674,...,2.287958,-0.081425,2.464453e+08,78.776587,16395.182098,0.495531,0.491789,49.717277,255.642147,0.295738
7,22000.0,0.007808,14.029445,32.638046,26.383538,0.284020,14.138076,0.000688,-0.092137,0.065338,...,1.076837,0.060912,2.460121e+08,75.056526,47710.482928,62.849083,62.850641,48.720865,222.107231,62.558301
8,23000.0,0.005119,14.023054,45.540919,22.319582,0.434587,15.443225,0.016146,-0.068099,-0.250712,...,1.062278,0.044513,2.457923e+08,72.028790,48192.713733,59.138337,59.139118,48.912070,231.078640,58.929745
9,24000.0,0.008985,14.039709,38.205425,21.048999,0.704119,14.068296,-0.004197,-0.077165,-0.201576,...,1.235466,0.028252,2.459489e+08,74.350943,59184.630333,78.430697,78.430600,48.412230,239.067010,78.030373


In [34]:
truckonetype =  pd.to_datetime(truck1data['Time (DateTime)'])

truckonetype.head(100)



16    2019-08-07 07:54:04.559
17    2019-08-07 07:54:04.659
18    2019-08-07 07:54:04.759
19    2019-08-07 07:54:04.859
20    2019-08-07 07:54:04.959
21    2019-08-07 07:54:05.059
22    2019-08-07 07:54:05.159
23    2019-08-07 07:54:05.259
24    2019-08-07 07:54:05.359
25    2019-08-07 07:54:05.459
26    2019-08-07 07:54:05.559
27    2019-08-07 07:54:05.659
28    2019-08-07 07:54:05.759
29    2019-08-07 07:54:05.859
30    2019-08-07 07:54:05.959
31    2019-08-07 07:54:06.059
32    2019-08-07 07:54:06.159
33    2019-08-07 07:54:06.259
34    2019-08-07 07:54:06.359
35    2019-08-07 07:54:06.459
36    2019-08-07 07:54:06.559
37    2019-08-07 07:54:06.659
38    2019-08-07 07:54:06.759
39    2019-08-07 07:54:06.859
40    2019-08-07 07:54:06.959
41    2019-08-07 07:54:07.059
42    2019-08-07 07:54:07.159
43    2019-08-07 07:54:07.259
44    2019-08-07 07:54:07.359
45    2019-08-07 07:54:07.459
                ...          
86    2019-08-07 07:54:11.559
87    2019-08-07 07:54:11.659
88    2019

In [ ]:
truckTwotype =  pd.to_datetime(truck2data['Time (DateTime)'])

truckTwotype.head(100)